# Generating Shakespeare

## Setup

We're going to download the collected plays of Shakespeare to use as our data.

Source: http://www.gutenberg.org/cache/epub/100/pg100.txt

The original source was preprocessed to remove sonnets and non-Shakesperean text added by Project Gutenberg.

In [42]:
import os

BASE_DIR = os.getcwd()
DATA_DIR = BASE_DIR + '/data/shakespeare/'

In [43]:
data = DATA_DIR + 'gutenberg_shakespeare_modified.txt' # preprocessed

with open(data, 'r') as f:
    text = f.read()
print('corpus length:', len(text))

('corpus length:', 5291227)


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

('total chars:', 88)


Sometimes it's useful to have a zero value in the dataset, e.g. for padding

In [4]:
chars.insert(0, "\0")

In [5]:
''.join(chars)

'\x00\n\r !"&\'(),-.0123456789:;<?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz|}\xbb\xbf\xef'

Map chars to indices and vice versa

In [6]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
print(char_indices)

{'\x00': 0, ' ': 3, '(': 8, ',': 10, '0': 13, '4': 17, '8': 21, '\xbb': 85, '<': 25, '\xbf': 86, 'D': 30, 'H': 34, 'L': 38, 'P': 42, 'T': 46, 'X': 50, '`': 56, 'd': 60, 'h': 64, 'l': 68, '\xef': 87, 'p': 72, 't': 76, 'x': 80, '|': 83, "'": 7, '3': 16, '7': 20, ';': 24, '?': 26, 'C': 29, 'G': 33, 'K': 37, 'O': 41, 'S': 45, 'W': 49, '[': 53, '_': 55, 'c': 59, 'g': 63, 'k': 67, 'o': 71, 's': 75, 'w': 79, '\n': 1, '"': 5, '&': 6, '.': 12, '2': 15, '6': 19, ':': 23, 'B': 28, 'F': 32, 'J': 36, 'N': 40, 'R': 44, 'V': 48, 'Z': 52, 'b': 58, 'f': 62, 'j': 66, 'n': 70, 'r': 74, 'v': 78, 'z': 82, '\r': 2, '!': 4, ')': 9, '-': 11, '1': 14, '5': 18, '9': 22, 'A': 27, 'E': 31, 'I': 35, 'M': 39, 'Q': 43, 'U': 47, 'Y': 51, ']': 54, 'a': 57, 'e': 61, 'i': 65, 'm': 69, 'q': 73, 'u': 77, 'y': 81, '}': 84}


*idx* converts the Shakepearean text to character indices (based on the *char_indices* mapping above)

In [8]:
idx = [char_indices[c] for c in text]

In [9]:
print(idx[:70])

[87, 85, 86, 45, 29, 31, 40, 31, 23, 2, 1, 44, 71, 77, 75, 65, 68, 68, 71, 70, 24, 3, 42, 57, 74, 65, 75, 24, 3, 32, 68, 71, 74, 61, 70, 59, 61, 24, 3, 39, 57, 74, 75, 61, 65, 68, 68, 61, 75, 2, 1, 2, 1, 2, 1, 27, 29, 46, 3, 35, 12, 3, 45, 29, 31, 40, 31, 3, 14, 12]


In [10]:
''.join(indices_char[i] for i in idx[:70])

'\xef\xbb\xbfSCENE:\r\nRousillon; Paris; Florence; Marseilles\r\n\r\n\r\nACT I. SCENE 1.'

## 3 char model

### Create inputs

Create a list of every 4th character, starting at the 0th, 1st, 2nd, then 3rd characters

In [11]:
nc=3 # num chars
c1_dat = [idx[i] for i in xrange(0, len(idx)-1-nc, nc)]
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-1-nc, nc)]
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-1-nc, nc)]
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-1-nc, nc)]

In [12]:
0, len(idx)-1-nc, nc

(0, 5291223, 3)

In [13]:
len(c1_dat), len(c4_dat)

(1763741, 1763741)

Out inputs

In [14]:
import numpy as np

x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)

Out output

In [15]:
y = np.stack(c4_dat)

In [16]:
x1.shape, y.shape

((1763741,), (1763741,))

In [17]:
n_fac = 42 # number of latent factors (size of embedding matrix)

Create inputs and embedding outputs for each of our 3 character inputs

In [18]:
from keras.layers import Input, Embedding
from keras.layers.core import Flatten

def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [19]:
c1_in, c1_emb = embedding_input('c1', vocab_size, n_fac)
c2_in, c2_emb = embedding_input('c2', vocab_size, n_fac)
c3_in, c3_emb = embedding_input('c3', vocab_size, n_fac)

### Create and train model

In [20]:
n_hidden = 256 # hyperparameter: size of hidden state

![3char](./3char.png)

`dense_in` is the 'green arrow' in the diagram - the layer operation from input to hidden

In [21]:
from keras.layers.core import Dense

dense_in = Dense(n_hidden, activation='relu')

Our first hidden activation is simply this function applied to the result of the embedding of the first character.

In [22]:
c1_hidden = dense_in(c1_emb)

`dense_hidden` is the 'orange arrow' from our diagram - the layer operation from hidden to hidden

_Note:_ unsure why the activation for this is `tanh`

In [23]:
dense_hidden = Dense(n_hidden, activation='tanh')

Our second and third activations sum up the previous hidden state (after applying `dense_hidden`) to the new input state.

In [24]:
from keras.layers import merge

# merge([new input state, orange arrow from previous hidden state])
c2_hidden = merge([dense_in(c2_emb), dense_hidden(c1_hidden)])
c3_hidden = merge([dense_in(c3_emb), dense_hidden(c2_hidden)])

`dense_out` is the 'blue arrow' from our diagram - the layer operation from hidden to output

In [25]:
dense_out = Dense(vocab_size, activation='softmax')

The third hidden state is the input to our output layer

In [26]:
c4_out = dense_out(c3_hidden)

In [27]:
from keras.models import Model
from keras.optimizers import Adam

model = Model([c1_in, c2_in, c3_in], c4_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.optimizer.lr=0.000001

In [28]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
c1 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
c2 (InputLayer)                  (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 42)         3696        c1[0][0]                         
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1, 42)         3696        c2[0][0]                         
___________________________________________________________________________________________

In [29]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
1763741/1763741 [==============================] - 127s - loss: 3.6654   
Epoch 2/4
1763741/1763741 [==============================] - 117s - loss: 3.1122   
Epoch 3/4
1763741/1763741 [==============================] - 120s - loss: 3.0255   
Epoch 4/4
1763741/1763741 [==============================] - 127s - loss: 2.9397   


In [30]:
model.optimizer.lr=0.01

In [31]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
1763741/1763741 [==============================] - 123s - loss: 2.8500   
Epoch 2/4
1763741/1763741 [==============================] - 134s - loss: 2.7623   
Epoch 3/4
1763741/1763741 [==============================] - 125s - loss: 2.6820   
Epoch 4/4
1763741/1763741 [==============================] - 125s - loss: 2.6122   


In [32]:
model.optimizer.lr=0.000001

In [33]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
1763741/1763741 [==============================] - 122s - loss: 2.5533   
Epoch 2/4
1763741/1763741 [==============================] - 132s - loss: 2.5044   
Epoch 3/4
1763741/1763741 [==============================] - 122s - loss: 2.4638   
Epoch 4/4
1763741/1763741 [==============================] - 125s - loss: 2.4301   


In [34]:
model.optimizer.lr=0.01

In [35]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
1763741/1763741 [==============================] - 120s - loss: 2.4018   
Epoch 2/4
1763741/1763741 [==============================] - 118s - loss: 2.3778   
Epoch 3/4
1763741/1763741 [==============================] - 129s - loss: 2.3571   
Epoch 4/4
1763741/1763741 [==============================] - 126s - loss: 2.3390   


In [36]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=10)

Epoch 1/10
1763741/1763741 [==============================] - 119s - loss: 2.3227   
Epoch 2/10
1763741/1763741 [==============================] - 124s - loss: 2.3080   
Epoch 3/10
1763741/1763741 [==============================] - 131s - loss: 2.2947   
Epoch 4/10
1763741/1763741 [==============================] - 134s - loss: 2.2824   
Epoch 5/10
1763741/1763741 [==============================] - 128s - loss: 2.2712   
Epoch 6/10
1763741/1763741 [==============================] - 125s - loss: 2.2609   
Epoch 7/10
1763741/1763741 [==============================] - 124s - loss: 2.2513   
Epoch 8/10
1763741/1763741 [==============================] - 121s - loss: 2.2425   
Epoch 9/10
1763741/1763741 [==============================] - 124s - loss: 2.2344   
Epoch 10/10
1763741/1763741 [==============================] - 125s - loss: 2.2268   


In [37]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=10)

Epoch 1/10
1763741/1763741 [==============================] - 133s - loss: 2.2197   
Epoch 2/10
1763741/1763741 [==============================] - 123s - loss: 2.2131   
Epoch 3/10
1763741/1763741 [==============================] - 126s - loss: 2.2068   
Epoch 4/10
1763741/1763741 [==============================] - 130s - loss: 2.2010   
Epoch 5/10
1763741/1763741 [==============================] - 128s - loss: 2.1955   
Epoch 6/10
1763741/1763741 [==============================] - 124s - loss: 2.1903   
Epoch 7/10
1763741/1763741 [==============================] - 124s - loss: 2.1853   
Epoch 8/10
1763741/1763741 [==============================] - 120s - loss: 2.1806   
Epoch 9/10
1763741/1763741 [==============================] - 124s - loss: 2.1761   
Epoch 10/10
1763741/1763741 [==============================] - 123s - loss: 2.1719   


In [38]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=10)

Epoch 1/10
1763741/1763741 [==============================] - 133s - loss: 2.1678   
Epoch 2/10
1763741/1763741 [==============================] - 127s - loss: 2.1639   
Epoch 3/10
1763741/1763741 [==============================] - 128s - loss: 2.1602   
Epoch 4/10
1763741/1763741 [==============================] - 130s - loss: 2.1566   
Epoch 5/10
1763741/1763741 [==============================] - 124s - loss: 2.1532   
Epoch 6/10
1763741/1763741 [==============================] - 125s - loss: 2.1498   
Epoch 7/10
1763741/1763741 [==============================] - 125s - loss: 2.1467   
Epoch 8/10
1763741/1763741 [==============================] - 133s - loss: 2.1436   
Epoch 9/10
1763741/1763741 [==============================] - 124s - loss: 2.1406   
Epoch 10/10
1763741/1763741 [==============================] - 128s - loss: 2.1378   


In [39]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=10)

Epoch 1/10
1763741/1763741 [==============================] - 124s - loss: 2.1350   
Epoch 2/10
1763741/1763741 [==============================] - 133s - loss: 2.1323   
Epoch 3/10
1763741/1763741 [==============================] - 123s - loss: 2.1298   
Epoch 4/10
1763741/1763741 [==============================] - 134s - loss: 2.1273   
Epoch 5/10
1763741/1763741 [==============================] - 125s - loss: 2.1248   
Epoch 6/10
1763741/1763741 [==============================] - 126s - loss: 2.1225   
Epoch 7/10
1763741/1763741 [==============================] - 127s - loss: 2.1202   
Epoch 8/10
1763741/1763741 [==============================] - 133s - loss: 2.1180   
Epoch 9/10
1763741/1763741 [==============================] - 122s - loss: 2.1158   
Epoch 10/10
1763741/1763741 [==============================] - 131s - loss: 2.1138   


Let's save the model.

In [44]:
model_path = DATA_DIR + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [45]:
save1_path = model_path + 'save1.h5'
if not os.path.exists(save1_path):
    model.save_weights(save1_path)
model.load_weights(save1_path)

### Test Model

"`newaxis` is used to increase the dimension of the existing array by one more dimension, when used once" - [source](https://stackoverflow.com/questions/29241056/the-use-of-numpy-newaxis)

In [46]:
def get_next(m, inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = m.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [47]:
get_next(model, 'phi')

's'

In [48]:
get_next(model, ' th')

'e'

In [49]:
get_next(model, ' an')

'd'

## Our first RNN!